In [2]:
"""
Visualize:
    * nand uncon before nand con cnt / nand con cnt
    * not uncon before not con cnt / not con cnt
    * any uncon before con cnt / con cnt 
    * nand_subopt_plastic_before_nand_opt_plastic_cnt / nand_opt_plastic_cnt
    * not_subopt_plastic_before_not_opt_plastic_cnt / not_opt_plastic_cnt
    * any_subopt_plastic_before_opt_plastic_cnt / any_opt_plastic_cnt  
"""

%matplotlib inline
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
# Load settings
settings_fpath = "../params/settings.json"
settings= None
with open(settings_fpath) as fp:
    settings = json.load(fp)
    
# Data file path
data_fpath = "/Users/amlalejini/Desktop/scratch/oopf16/processed/lineage_stepping_stones_overview__2016-11-16.csv"

df = pd.read_csv(data_fpath)

#### Some manual settings
env_order = [u'ntasks_1__envs_1__tasks_NAND',
             u'ntasks_1__envs_1__tasks_NOT',
             u'ntasks_2__envs_1__tasks_NAND_NOT',
             u'ntasks_1__envs_2__tasks_NAND',
             u'ntasks_1__envs_2__tasks_NOT',
             u'ntasks_2__envs_2__tasks_NAND_NOT',
             u'ntasks_2__envs_4__tasks_NAND_NOT',
             u'ntasks_2__envs_4__tasks_NOT_NAND',
             u'ntasks_2__envs_4__tasks_NAND_NOT__style_2',
             u'ntasks_2__envs_4__tasks_NOT_NAND__style_2']


In [10]:
##########################
# Helper functions
##########################
def GetTreatmentAttr(treatment, attr):
    """
    Given a treatment name and an attribute, what are its attributes?
    """
    attrs = {attr.split("_")[0]:attr.split("_")[1:] for attr in treatment.split("__")}
    return attrs[attr]

def GetTreatmentEnvType(treatment):
    return settings["treatments"][treatment]["settings"]

##########################
# Data Organization
##########################
# Make a list of treatments by change rate. 
all_treatments = set([])
treatments_by_change_rate = {}
treatments_by_env = {}
change_rates = set([])
for treatment in df['treatment']:
    all_treatments.add(treatment)
    # First, get change rate for this treatment.
    change_rate = int(GetTreatmentAttr(treatment, "cr")[0])
    change_rates.add(change_rate)
    if not change_rate in treatments_by_change_rate:
        treatments_by_change_rate[change_rate] = set()
    treatments_by_change_rate[change_rate].add(treatment)
    # Next, sort by environment type. 
    env = GetTreatmentEnvType(treatment)
    if not env in treatments_by_env:
        treatments_by_env[env] = set()
    treatments_by_env[env].add(treatment)

# Sort treatments_by_env by change rate.
for env in treatments_by_env:
    treatments_by_env[env] = list(treatments_by_env[env])
    treatments_by_env[env].sort(key = lambda treatment: int(GetTreatmentAttr(treatment, "cr")[0]))

    

In [11]:
##########################
# By environment type, plot:
#  * nand uncon before nand con / nand con count
##########################
for env in env_order:
    treatments = treatments_by_env[env]
    for treatment in treatments:
        print "Treatment: %s" % treatment
        num = df[df.treatment == treatment]["nand_uncon_before_nand_con_cnt"].iloc[0]
        den = df[df.treatment == treatment]["nand_con_cnt"].iloc[0]
        frac = 0
        if den != 0: frac = float(num) / float(den)
        print "  nand uncon before con/nand con: %d/%d (%f)" % (num, den, frac)
        num = df[df.treatment == treatment]["not_uncon_before_not_con_cnt"].iloc[0]
        den = df[df.treatment == treatment]["not_con_cnt"].iloc[0]
        frac = 0
        if den != 0: frac = float(num) / float(den)
        print "  not uncon before con/not con: %d/%d (%f)" % (num, den, frac)
        num = df[df.treatment == treatment]["any_uncon_before_con_cnt"].iloc[0]
        den = df[df.treatment == treatment]["con_cnt"].iloc[0]
        frac = 0
        if den != 0: frac = float(num) / float(den)
        print "  any uncon before con/any con: %d/%d (%f)" % (num, den, frac)
        num = df[df.treatment == treatment]["nand_subopt_plastic_before_nand_opt_plastic_cnt"].iloc[0]
        den = df[df.treatment == treatment]["nand_opt_plastic_cnt"].iloc[0]
        frac = 0
        if den != 0: frac = float(num) / float(den)
        print "  nand subopt plast before opt plast/nand opt plast: %d/%d (%f)" % (num, den, frac)
        num = df[df.treatment == treatment]["not_subopt_plastic_before_not_opt_plastic_cnt"].iloc[0]
        den = df[df.treatment == treatment]["not_opt_plastic_cnt"].iloc[0]
        frac = 0
        if den != 0: frac = float(num) / float(den)
        print "  not subopt plast before opt plast/not opt plast: %d/%d (%f)" % (num, den, frac)
        num = df[df.treatment == treatment]["any_subopt_plastic_before_opt_plastic_cnt"].iloc[0]
        den = df[df.treatment == treatment]["any_opt_plastic_cnt"].iloc[0]
        frac = 0
        if den != 0: frac = float(num) / float(den)
        print "  any subopt plast before opt plast/any opt plast: %d/%d (%f)" % (num, den, frac)
# fig, axes = plt.subplots(nrows = len(by_environment.keys()) - 3, ncols = 1, figsize = (16, 32))
# axi = 0
# y_range = [i for i in range(0, 51, 10)]


Treatment: ntasks_1__envs_1__cr_0__tasks_NAND__mr_p0075
  nand uncon before con/nand con: 0/0 (0.000000)
  not uncon before con/not con: 0/0 (0.000000)
  any uncon before con/any con: 0/0 (0.000000)
  nand subopt plast before opt plast/nand opt plast: 0/0 (0.000000)
  not subopt plast before opt plast/not opt plast: 0/0 (0.000000)
  any subopt plast before opt plast/any opt plast: 0/0 (0.000000)
Treatment: ntasks_1__envs_1__cr_0__tasks_NOT__mr_p0075
  nand uncon before con/nand con: 0/0 (0.000000)
  not uncon before con/not con: 0/0 (0.000000)
  any uncon before con/any con: 0/0 (0.000000)
  nand subopt plast before opt plast/nand opt plast: 0/0 (0.000000)
  not subopt plast before opt plast/not opt plast: 0/0 (0.000000)
  any subopt plast before opt plast/any opt plast: 0/0 (0.000000)
Treatment: ntasks_2__envs_1__cr_0__tasks_NAND_NOT__mr_p0075
  nand uncon before con/nand con: 0/0 (0.000000)
  not uncon before con/not con: 0/0 (0.000000)
  any uncon before con/any con: 0/0 (0.000000)


In [7]:
treatments_by_env.keys()

[u'ntasks_2__envs_4__tasks_NAND_NOT',
 u'ntasks_2__envs_2__tasks_NAND_NOT',
 u'ntasks_1__envs_2__tasks_NAND',
 u'ntasks_1__envs_1__tasks_NAND',
 u'ntasks_2__envs_1__tasks_NAND_NOT',
 u'ntasks_1__envs_2__tasks_NOT',
 u'ntasks_2__envs_4__tasks_NAND_NOT__style_2',
 u'ntasks_2__envs_4__tasks_NOT_NAND__style_2',
 u'ntasks_2__envs_4__tasks_NOT_NAND',
 u'ntasks_1__envs_1__tasks_NOT']